In [100]:
import tensorflow as tf
import random
import numpy as np
import matplotlib.image as mpimg
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.examples.tutorials.mnist import input_data


In [101]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
image_pixels = 28
kernel = 5
sess = tf.InteractiveSession()

Extracting dataset/train-images-idx3-ubyte.gz
Extracting dataset/train-labels-idx1-ubyte.gz
Extracting dataset/t10k-images-idx3-ubyte.gz
Extracting dataset/t10k-labels-idx1-ubyte.gz


In [102]:
def weight_initialise(shape):
    value = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(value)

In [103]:
def bias_initialise(shape):
    value = tf.constant(0.1, shape=shape)
    return tf.Variable(value)

In [104]:
def classifier(x):
   
    input_layer = tf.reshape(x, [-1, image_pixels, image_pixels, 1])

    params_conv1 = weight_initialise([kernel, kernel, 1, 32])

    bias_conv1 = bias_initialise([32])

    output_conv1 = tf.nn.relu(tf.nn.conv2d(input_layer, params_conv1, strides=[1, 1, 1, 1], padding='SAME')+bias_conv1)

    pool_conv1 = tf.nn.max_pool(output_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
  
    params_conv2 = weight_initialise([kernel, kernel, 32, 64])
    bias_conv2 = bias_initialise([64])

    output_conv2 = tf.nn.relu(tf.nn.conv2d(pool_conv1, params_conv2, strides=[1, 1, 1, 1], padding='SAME')+bias_conv2)

    pool_conv2 = tf.nn.max_pool(output_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    params_fc1 = weight_initialise([7*7*64, 1024])
    bias_fc1 = bias_initialise([1024])
    pool_conv2_flat = tf.reshape(pool_conv2, [-1, 7*7*64])
    output_fc1 = tf.nn.relu(tf.matmul(pool_conv2_flat, params_fc1) + bias_fc1)

    keep_prob = tf.placeholder(tf.float32)
    dropout = tf.nn.dropout(output_fc1, keep_prob)

    params_fc2 = weight_initialise([1024, 10])
    bias_fc2 = bias_initialise([10])
    y_conv = tf.matmul(dropout, params_fc2) + bias_fc2

    return y_conv, keep_prob

In [105]:
# Declaring variables

x = tf.placeholder(tf.float32, [None, 784])
targets = tf.placeholder(tf.float32, [None, 10])
y_conv, keep_prob = classifier(x)

In [106]:
y = tf.nn.softmax(y_conv)

In [107]:
# Calculating loss

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=y_conv)
cross_entropy = tf.reduce_mean(cross_entropy)


In [108]:
train = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(targets, 1))
correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)

In [109]:
sess.run(tf.global_variables_initializer())

In [78]:
for i in range(1000):
    batch = mnist.train.next_batch(100)
    if i%200 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], targets: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))

    train.run(feed_dict={x: batch[0], targets: batch[1], keep_prob: 0.4})

step 0, training accuracy 0.08
step 200, training accuracy 0.9
step 400, training accuracy 0.98
step 600, training accuracy 1
step 800, training accuracy 0.95


In [81]:
print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images[0:500], 
                                                  targets: mnist.test.labels[0:500], keep_prob: 1.0}))

test accuracy 0.986
